# SEGMENTING AND CLUSTERING NEIGHBORHOODS IN TORONTO
In this workbook, we will explore, segment, and cluster the neighborhoods in the city of Toronto. The neighborhood data is not readily available on the internet,therefore, we'll be required to scrape the data found on a Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format. 

Once the data is in a structured format, we'll use the Foursquare API to explore neighborhoods in Toronto. we'll use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. we'll use the k-means clustering algorithm to complete this task. To finish off, we'll use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

## Importing necessary Libraries
We start by importing the libraries that are required to carry out the assignment.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from IPython.display import display_html
from bs4 import BeautifulSoup

## Retrieval of Canadian Postal Codes
We'll be using the BeautifulSoup Library of Python for web scraping the Wikipedia page in order to retrieve the table we need, which contains the postal codes. The title of the webpage is printed to check if the page has been scraped successfully or not. Futhermore, we print the table to verify it's content.

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
table = str(soup.table)
display_html(table,raw=True)

## Converting table to Pandas Dataframe
We convert the table to a pandas dataframe for cleaning and processing the data at hand.

In [ ]:
dfs = pd.read_html(table)
df=dfs[0]
df.head()

## Data Cleaning and Preprocessing
In this assignment, we've been ordered to carry out the following instructions regarding data cleaning and preprocessing:
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
# Dropping the rows where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Combining the neighbourhoods with same Postalcode
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2

In [ ]:
# Shape of data frame
df2.shape

## Importing the file containing the geographical coordinates of each postal code

In [ ]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

## Merging the two tables into one 
Now we'll merge the "df2" table with the "lat_lon" table into one using the "Postal Code" column as reference.

In [ ]:
df3 = pd.merge(df2,lat_lon,on='Postal Code')
df3.head()

## Selecting only Boroughs located in Toronto

In [ ]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

## Visualizing the neighborhoods using Folium
Now we'll create the folium map displaying the neighborhoods shown in the table above.

In [ ]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

## Using the Kmeans algorithm to cluster the neighborhoods
Now we'll use the kmeans algorithm to cluster the neighborhoods.

In [ ]:
k=5
toronto_clustering = df4.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
df4

## Displaying the clusters in the map

In [ ]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters